In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
input_ids = tokenizer("Just an example", add_special_tokens=True, return_tensors="pt").input_ids
pt_outputs = model(input_ids)

Uncomment the below line to see the tensor ouput (WARNING : IT'S A LARGE TENSOR)

In [ ]:
#print(pt_outputs)

In [ ]:
torch.onnx.export(model,
                 (input_ids),
                 "bert-ner.onnx",
                 input_names=["input_ids"],
                 output_names=["outputs"],
                 opset_version=12)

In [ ]:
!python3 /opt/intel/openvino_2021.2.200/deployment_tools/model_optimizer/mo.py --input_model bert-ner.onnx

In [ ]:
from openvino.inference_engine import IECore

In [ ]:
ie = IECore()

In [ ]:
openvino_bert = ie.read_network(model="bert-ner.xml", weights="bert-ner.bin")
exec_bert = ie.load_network(network=openvino_bert, device_name="CPU", num_requests=1)
openvino_outputs = exec_bert.infer(inputs={"input_ids": input_ids})

In [ ]:
print(openvino_outputs)